## Линейная и полиномиальная регрессия для определения цен на тыквы - Урок 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Инфографика от Дасани Мадипалли</figcaption>


<!--![Инфографика от Дасани Мадипалли](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.ru.png){width="800"}-->

#### Введение

До этого момента вы изучили, что такое регрессия, используя пример данных из набора данных о ценах на тыквы, который мы будем использовать на протяжении всего урока. Вы также визуализировали данные с помощью `ggplot2`.💪

Теперь вы готовы углубиться в регрессию для машинного обучения. В этом уроке вы узнаете больше о двух типах регрессии: *основной линейной регрессии* и *полиномиальной регрессии*, а также о некоторых математических основах этих методов.

> На протяжении всей программы мы предполагаем минимальные знания математики и стремимся сделать материал доступным для студентов из других областей. Следите за заметками, 🧮 подсказками, диаграммами и другими инструментами обучения, которые помогут вам понять материал.

#### Подготовка

Напомним, что вы загружаете эти данные, чтобы задавать вопросы.

-   Когда лучше всего покупать тыквы?

-   Какую цену можно ожидать за ящик миниатюрных тыкв?

-   Стоит ли покупать их в корзинах на пол-бушеля или в коробках на 1 1/9 бушеля? Давайте продолжим исследовать эти данные.

В предыдущем уроке вы создали `tibble` (современное переосмысление фрейма данных) и заполнили его частью исходного набора данных, стандартизировав цены по бушелю. Однако, таким образом, вы смогли собрать только около 400 точек данных и только за осенние месяцы. Может быть, мы сможем получить немного больше информации о природе данных, если очистим их лучше? Посмотрим... 🕵️‍♀️

Для выполнения этой задачи нам понадобятся следующие пакеты:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) — это [коллекция пакетов для R](https://www.tidyverse.org/packages), разработанная для того, чтобы сделать работу с данными быстрее, проще и увлекательнее!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) — это [фреймворк](https://www.tidymodels.org/packages), состоящий из пакетов для моделирования и машинного обучения.

-   `janitor`: [пакет janitor](https://github.com/sfirke/janitor) предоставляет простые инструменты для анализа и очистки "грязных" данных.

-   `corrplot`: [пакет corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) предлагает визуальный инструмент для исследования матрицы корреляции, поддерживающий автоматическую перестановку переменных для выявления скрытых закономерностей между ними.

Вы можете установить их следующим образом:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Скрипт ниже проверяет, установлены ли у вас необходимые пакеты для выполнения этого модуля, и устанавливает их, если они отсутствуют.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Мы позже загрузим эти замечательные пакеты и сделаем их доступными в нашей текущей R-сессии. (Это просто для иллюстрации, `pacman::p_load()` уже сделал это за вас)

## 1. Линия линейной регрессии

Как вы узнали в Уроке 1, цель упражнения по линейной регрессии — построить *линию* *наилучшего соответствия*, чтобы:

-   **Показать взаимосвязь переменных**. Показать связь между переменными.

-   **Сделать прогнозы**. Делать точные прогнозы о том, где новая точка данных окажется относительно этой линии.

Для построения такой линии мы используем статистический метод, называемый **регрессией наименьших квадратов**. Термин `наименьшие квадраты` означает, что все точки данных вокруг линии регрессии возводятся в квадрат, а затем складываются. В идеале, итоговая сумма должна быть как можно меньше, потому что мы хотим минимизировать количество ошибок, или `наименьшие квадраты`. Таким образом, линия наилучшего соответствия — это линия, которая дает нам наименьшее значение суммы квадратов ошибок — отсюда и название *регрессия наименьших квадратов*.

Мы делаем это, потому что хотим смоделировать линию, которая имеет минимальное суммарное расстояние от всех наших точек данных. Мы также возводим значения в квадрат перед их сложением, так как нас интересует их величина, а не направление.

> **🧮 Покажите мне математику**
>
> Эта линия, называемая *линией наилучшего соответствия*, может быть выражена [уравнением](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` — это '`объясняющая переменная` или `предиктор`'. `Y` — это '`зависимая переменная` или `результат`'. Наклон линии обозначается как `b`, а `a` — это точка пересечения с осью Y, которая указывает значение `Y`, когда `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "наклон = $y/x$")
    Инфографика от Jen Looper
>
> Сначала вычислите наклон `b`.
>
> Другими словами, если обратиться к исходному вопросу о данных по тыквам: "предсказать цену тыквы за бушель по месяцам", `X` будет означать цену, а `Y` — месяц продажи.
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.ru.png)
    Инфографика от Jen Looper
> 
> Вычислите значение Y. Если вы платите около \$4, значит, это апрель!
>
> Математика, которая рассчитывает линию, должна показать наклон линии, который также зависит от точки пересечения, или от того, где находится `Y`, когда `X = 0`.
>
> Вы можете ознакомиться с методом расчета этих значений на сайте [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Также посетите [этот калькулятор наименьших квадратов](https://www.mathsisfun.com/data/least-squares-calculator.html), чтобы увидеть, как значения чисел влияют на линию.

Не так уж страшно, правда? 🤓

#### Корреляция

Еще один термин, который нужно понять, — это **коэффициент корреляции** между заданными переменными X и Y. Используя диаграмму рассеяния, вы можете быстро визуализировать этот коэффициент. График с точками данных, расположенными в аккуратной линии, имеет высокую корреляцию, а график с точками данных, разбросанными повсюду между X и Y, имеет низкую корреляцию.

Хорошая модель линейной регрессии — это та, которая имеет высокий коэффициент корреляции (ближе к 1, чем к 0) с использованием метода регрессии наименьших квадратов и линии регрессии.


## **2. Танец с данными: создание датафрейма для моделирования**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Иллюстрация @allison_horst</figcaption>


<!--![Иллюстрация \@allison_horst](../../../../../../translated_images/janitor.e4a77dd3d3e6a32e25327090b8a9c00dc7cf459c44fa9f184c5ecb0d48ce3794.ru.jpg){width="700"}-->


Загрузите необходимые библиотеки и набор данных. Преобразуйте данные в датафрейм, содержащий подмножество данных:

-   Оставьте только тыквы, цена которых указана за бушель

-   Преобразуйте дату в месяц

-   Рассчитайте цену как среднее значение между высокой и низкой ценами

-   Преобразуйте цену, чтобы она отражала стоимость за количество в бушелях

> Мы рассмотрели эти шаги в [предыдущем уроке](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

В духе чистого приключения давайте изучим [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), который предоставляет простые функции для анализа и очистки грязных данных. Например, давайте посмотрим на названия столбцов в наших данных:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Мы можем сделать лучше. Давайте сделаем эти названия столбцов `friendR`, преобразовав их в [snake_case](https://en.wikipedia.org/wiki/Snake_case) с помощью `janitor::clean_names`. Чтобы узнать больше об этой функции: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Много tidyR 🧹! Теперь танец с данными, используя `dplyr`, как в предыдущем уроке! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Отличная работа!👌 Теперь у вас есть чистый, аккуратный набор данных, на основе которого можно построить новую регрессионную модель!

Как насчет диаграммы рассеяния?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Диаграмма рассеяния напоминает нам, что у нас есть данные только за месяцы с августа по декабрь. Вероятно, нам нужно больше данных, чтобы сделать выводы в линейной форме.

Давайте снова посмотрим на наши данные для моделирования:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Что, если мы захотим предсказать `price` тыквы на основе столбцов `city` или `package`, которые имеют тип "символы"? Или, еще проще, как мы могли бы найти корреляцию (которая требует, чтобы оба входных значения были числовыми) между, например, `package` и `price`? 🤷🤷

Модели машинного обучения лучше работают с числовыми признаками, чем с текстовыми значениями, поэтому обычно необходимо преобразовать категориальные признаки в числовые представления.

Это означает, что нам нужно найти способ преобразовать наши предикторы, чтобы модель могла эффективно их использовать. Этот процесс называется `feature engineering`.


## 3. Предобработка данных для моделирования с помощью recipes 👩‍🍳👨‍🍳

Действия, которые преобразуют значения предикторов, чтобы сделать их более удобными для использования моделью, называются `feature engineering` (инженерия признаков).

Разные модели имеют разные требования к предобработке. Например, метод наименьших квадратов требует `кодирования категориальных переменных`, таких как месяц, сорт и название города. Это просто означает `преобразование` столбца с `категориальными значениями` в один или несколько `числовых столбцов`, которые заменяют оригинальный.

Например, предположим, что ваши данные включают следующую категориальную переменную:

|  город   |
|:--------:|
| Денвер   |
| Найроби  |
| Токио    |

Вы можете применить *порядковое кодирование*, чтобы заменить каждую категорию уникальным целым числом, например так:

| город |
|:-----:|
|   0   |
|   1   |
|   2   |

Именно это мы и сделаем с нашими данными!

В этом разделе мы изучим еще один удивительный пакет из экосистемы Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - который предназначен для помощи в предобработке данных **до** обучения модели. В своей основе рецепт (recipe) - это объект, который определяет, какие шаги должны быть применены к набору данных, чтобы подготовить его для моделирования.

Теперь давайте создадим рецепт, который подготовит наши данные для моделирования, заменяя уникальным целым числом все наблюдения в столбцах предикторов:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Отлично! 👏 Мы только что создали наш первый рецепт, который задает результат (цена) и соответствующие предикторы, а также указывает, что все столбцы предикторов должны быть закодированы в набор целых чисел 🙌! Давайте быстро разберем, как это работает:

-   Вызов `recipe()` с формулой указывает рецепту *роли* переменных, используя данные `new_pumpkins` в качестве эталона. Например, столбец `price` был назначен как `outcome` (результат), в то время как остальные столбцы были назначены как `predictor` (предикторы).

-   `step_integer(all_predictors(), zero_based = TRUE)` указывает, что все предикторы должны быть преобразованы в набор целых чисел, начиная с 0.

Мы уверены, что у вас могут возникнуть мысли вроде: "Это так круто!! Но что, если мне нужно убедиться, что рецепты действительно делают то, что я от них ожидаю? 🤔"

Это отличная мысль! Видите ли, как только ваш рецепт определен, вы можете оценить параметры, необходимые для предварительной обработки данных, а затем извлечь обработанные данные. Обычно вам не нужно делать это при использовании Tidymodels (мы скоро увидим стандартный подход —\> `workflows`), но это может быть полезно, если вы хотите провести проверку, чтобы убедиться, что рецепты работают так, как вы ожидаете.

Для этого вам понадобятся еще два действия: `prep()` и `bake()`. И, как всегда, наши маленькие друзья из R от [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) помогут вам лучше понять это!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Иллюстрация от @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): оценивает необходимые параметры на основе обучающего набора данных, которые затем могут быть применены к другим наборам данных. Например, для заданного столбца предиктора, какое наблюдение будет назначено целому числу 0, 1, 2 и т.д.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): берет подготовленный рецепт и применяет операции к любому набору данных.

Итак, давайте подготовим и применим наши рецепты, чтобы действительно убедиться, что "под капотом" столбцы предикторов сначала будут закодированы, прежде чем модель будет обучена.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Ура! 🥳 Обработанные данные `baked_pumpkins` имеют все предикторы закодированными, что подтверждает, что этапы предварительной обработки, определенные в нашем рецепте, действительно работают как задумано. Это делает данные сложнее для чтения, но гораздо более понятными для Tidymodels! Потратьте немного времени, чтобы выяснить, какое наблюдение было сопоставлено с соответствующим целым числом.

Также стоит упомянуть, что `baked_pumpkins` — это фрейм данных, с которым мы можем выполнять вычисления.

Например, давайте попробуем найти хорошую корреляцию между двумя точками ваших данных, чтобы потенциально построить хорошую предсказательную модель. Для этого мы воспользуемся функцией `cor()`. Введите `?cor()`, чтобы узнать больше об этой функции.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Как оказалось, между Городом и Ценой существует лишь слабая корреляция. Однако между Упаковкой и её Ценой корреляция немного сильнее. Логично, правда? Обычно, чем больше коробка с продуктами, тем выше цена.

Раз уж мы этим занимаемся, давайте также попробуем визуализировать матрицу корреляции всех столбцов с помощью пакета `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Намного лучше.

Хороший вопрос, который теперь можно задать этим данным: '`Какую цену я могу ожидать для заданного пакета тыкв?`' Давайте разберемся!

> Примечание: Когда вы **`bake()`** подготовленный рецепт **`pumpkins_prep`** с параметром **`new_data = NULL`**, вы извлекаете обработанные (т.е. закодированные) тренировочные данные. Если у вас был бы другой набор данных, например, тестовый набор, и вы хотели бы увидеть, как рецепт его предварительно обработает, вы просто запекаете **`pumpkins_prep`** с параметром **`new_data = test_set`**.

## 4. Построение модели линейной регрессии

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Инфографика от Дасани Мадипалли</figcaption>


<!--![Инфографика от Дасани Мадипалли](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.ru.png){width="800"}-->


Теперь, когда мы создали рецепт и убедились, что данные будут предварительно обработаны должным образом, давайте построим регрессионную модель, чтобы ответить на вопрос: `Какую цену я могу ожидать для данного пакета тыквы?`

#### Обучение модели линейной регрессии с использованием обучающего набора данных

Как вы, возможно, уже догадались, колонка *price* является переменной `результата`, а колонка *package* — переменной `предиктором`.

Для этого мы сначала разделим данные так, чтобы 80% попало в обучающий набор, а 20% — в тестовый. Затем определим рецепт, который закодирует колонку предиктора в набор целых чисел, и создадим спецификацию модели. Мы не будем подготавливать и применять наш рецепт, так как уже знаем, что он обработает данные должным образом.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Отличная работа! Теперь, когда у нас есть рецепт и спецификация модели, нам нужно найти способ объединить их в один объект, который сначала будет предварительно обрабатывать данные (подготовка + выпекание за кулисами), обучать модель на предварительно обработанных данных, а также предоставлять возможность для потенциальной постобработки. Как вам такое для спокойствия?🤩

В Tidymodels этот удобный объект называется [`workflow`](https://workflows.tidymodels.org/) и удобно хранит ваши компоненты моделирования! Это то, что мы бы назвали *конвейерами* (*pipelines*) в *Python*.

Итак, давайте объединим всё в workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Вдобавок, рабочий процесс может быть настроен/обучен почти так же, как и модель.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Из результатов модели мы можем увидеть коэффициенты, полученные в процессе обучения. Они представляют собой коэффициенты линии наилучшего соответствия, которая минимизирует общую ошибку между фактической и предсказанной переменной.

#### Оценка производительности модели с использованием тестового набора

Пришло время проверить, как справилась модель 📏! Как это сделать?

Теперь, когда мы обучили модель, мы можем использовать её для предсказаний на `test_set` с помощью `parsnip::predict()`. Затем мы сравним эти предсказания с фактическими значениями меток, чтобы оценить, насколько хорошо (или не очень!) работает модель.

Начнем с предсказаний для тестового набора, а затем объединим столбцы с тестовым набором.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Да, вы только что обучили модель и использовали её для предсказаний! 🔮 Хороша ли она? Давайте оценим производительность модели!

В Tidymodels мы делаем это с помощью `yardstick::metrics()`! Для линейной регрессии давайте сосредоточимся на следующих метриках:

-   `Root Mean Square Error (RMSE)` (Среднеквадратичная ошибка): Квадратный корень из [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Это абсолютная метрика в тех же единицах измерения, что и целевая переменная (в данном случае, цена тыквы). Чем меньше значение, тем лучше модель (в упрощённом смысле, это средняя цена, на которую предсказания ошибаются!).

-   `Coefficient of Determination` (обычно известен как R-квадрат или R2): Относительная метрика, где чем выше значение, тем лучше модель соответствует данным. По сути, эта метрика показывает, какую долю дисперсии между предсказанными и фактическими значениями целевой переменной модель способна объяснить.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Там идет производительность модели. Давайте посмотрим, сможем ли мы получить лучшее представление, визуализировав диаграмму рассеяния с пакетом и ценой, а затем наложив линию наилучшего соответствия, используя сделанные прогнозы.

Это означает, что нам нужно подготовить и обработать тестовый набор, чтобы закодировать столбец пакета, а затем связать его с прогнозами, сделанными нашей моделью.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Отлично! Как вы можете видеть, модель линейной регрессии не очень хорошо обобщает связь между упаковкой и её соответствующей ценой.

🎃 Поздравляем, вы только что создали модель, которая может помочь предсказать цену нескольких видов тыкв. Ваш праздничный тыквенный огород будет великолепным. Но, вероятно, вы можете создать модель получше!

## 5. Построение модели полиномиальной регрессии

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Инфографика от Дасани Мадипалли</figcaption>


<!--![Инфографика от Дасани Мадипалли](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.ru.png){width="800"}-->


Иногда наши данные могут не иметь линейной зависимости, но мы все равно хотим предсказать результат. Полиномиальная регрессия может помочь нам делать прогнозы для более сложных нелинейных зависимостей.

Возьмем, например, зависимость между упаковкой и ценой в нашем наборе данных о тыквах. Хотя иногда между переменными существует линейная зависимость — чем больше объем тыквы, тем выше цена — иногда такие зависимости нельзя изобразить в виде плоскости или прямой линии.

> ✅ Вот [несколько примеров](https://online.stat.psu.edu/stat501/lesson/9/9.8) данных, для которых может быть полезна полиномиальная регрессия
>
> Взгляните еще раз на зависимость между сортом и ценой на предыдущем графике. Кажется ли, что этот диаграмма рассеяния обязательно должна быть проанализирована с помощью прямой линии? Возможно, нет. В таком случае можно попробовать полиномиальную регрессию.
>
> ✅ Полиномы — это математические выражения, которые могут состоять из одной или нескольких переменных и коэффициентов.

#### Обучение модели полиномиальной регрессии с использованием обучающего набора данных

Полиномиальная регрессия создает *изогнутую линию*, чтобы лучше описать нелинейные данные.

Давайте посмотрим, сможет ли полиномиальная модель лучше справиться с прогнозированием. Мы будем следовать примерно той же процедуре, что и раньше:

-   Создать рецепт, который определяет шаги предварительной обработки данных, чтобы подготовить их для моделирования, например: кодирование предикторов и вычисление полиномов степени *n*

-   Построить спецификацию модели

-   Объединить рецепт и спецификацию модели в рабочий процесс

-   Создать модель, подогнав рабочий процесс

-   Оценить, насколько хорошо модель работает на тестовых данных

Давайте начнем!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Оценка производительности модели

👏👏Вы создали полиномиальную модель, давайте сделаем прогнозы на тестовом наборе данных!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Ура, давайте оценим, как модель справилась с test_set, используя `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Намного лучшее качество работы.

`rmse` снизился с примерно 7 до примерно 3, что указывает на уменьшение ошибки между фактической ценой и предсказанной ценой. Можно *условно* интерпретировать это как среднюю ошибку в прогнозах около \$3. `rsq` увеличился с примерно 0.4 до 0.8.

Все эти метрики показывают, что полиномиальная модель работает значительно лучше, чем линейная. Отличная работа!

Давайте посмотрим, сможем ли мы это визуализировать!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Вы можете увидеть кривую, которая лучше подходит к вашим данным! 🤩

Вы можете сделать её более плавной, передав полиномиальную формулу в `geom_smooth`, например так:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Похоже на плавную кривую!🤩

Вот как можно сделать новый прогноз:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Прогнозирование с использованием `полиномиальной модели` действительно имеет смысл, учитывая диаграммы рассеяния для `price` и `package`! И если эта модель лучше предыдущей, глядя на те же данные, вам нужно заложить бюджет на эти более дорогие тыквы!

🏆 Отличная работа! Вы создали две регрессионные модели за один урок. В последнем разделе о регрессии вы узнаете о логистической регрессии для определения категорий.

## **🚀Задание**

Протестируйте несколько различных переменных в этом ноутбуке, чтобы увидеть, как корреляция влияет на точность модели.

## [**Викторина после лекции**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Обзор и самостоятельное изучение**

На этом уроке мы изучили линейную регрессию. Существуют и другие важные типы регрессии. Прочитайте о методах Stepwise, Ridge, Lasso и Elasticnet. Хороший курс для изучения этих методов — [курс Стэнфорда по статистическому обучению](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Если вы хотите узнать больше о том, как использовать удивительный фреймворк Tidymodels, ознакомьтесь со следующими ресурсами:

-   Сайт Tidymodels: [Начните с Tidymodels](https://www.tidymodels.org/start/)

-   Макс Кун и Джулия Силге, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **СПАСИБО:**

[Эллисон Хорст](https://twitter.com/allison_horst?lang=en) за создание удивительных иллюстраций, которые делают R более доступным и увлекательным. Найдите больше иллюстраций в её [галерее](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникающие в результате использования данного перевода.
